In [ ]:
from deepface import DeepFace
import cv2
import os
import datetime
import platform

def is_gui_available():
    return platform.system() != "Linux" or os.environ.get("DISPLAY") is not None

def analyze_and_draw(frame):
    try:
        results = DeepFace.analyze(frame, actions=["age", "gender", "emotion"], enforce_detection=False)
        y_label = 30  
        for face in results:
            region = face.get("region", {})
            if not region:
                continue
            x = region.get("x", 0)
            y = region.get("y", 0)
            w = region.get("w", 0)
            h = region.get("h", 0)

            gender_data = face.get("gender", {})
            gender = max(gender_data, key=gender_data.get) if isinstance(gender_data, dict) else str(gender_data)
            age = str(face.get("age", "?"))
            emotion = str(face.get("dominant_emotion", "?"))
            label = f"Gender: {gender} | Age: {age} | Emotion: {emotion}"

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (10, y_label), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            y_label += 30 
    except Exception as e:
        print("Lỗi phân tích gương mặt:", e)
    return frame

def create_output_dir():
    os.makedirs("result_deepface", exist_ok=True)
    return "result_deepface"

def analyze_from_image(image_path):
    if not os.path.exists(image_path):
        print("Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    result = analyze_and_draw(image)

    if is_gui_available():
        cv2.imshow("Result from image", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    output_path = os.path.join(create_output_dir(), f"deepface_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, result)
    print(f" Ảnh kết quả đã lưu tại: {output_path}")

def analyze_from_video(video_path):
    if not os.path.exists(video_path):
        print("Không tìm thấy video:", video_path)
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Không mở được video.")
        return

    output_dir = create_output_dir()
    last_result = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        result = analyze_and_draw(frame)
        last_result = result.copy()

        if is_gui_available():
            cv2.imshow("DeepFace video", result)

        if is_gui_available() and cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()

    if last_result is not None:
        filename = f"deepface_video_lastframe_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, last_result)
        print(f" Đã lưu frame cuối cùng từ video: {output_path}")
    print("Đã kết thúc phân tích video.")

def analyze_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở webcam.")
        return

    output_dir = create_output_dir()
    analyzed_frame = None

    print("📡 Đang phân tích từ webcam... Nhấn 'e' hoặc 'ESC' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        try:
            results = DeepFace.analyze(frame, actions=["age", "gender", "emotion"], enforce_detection=False)
            y_label = 30
            for face in results:
                region = face.get("region", {})
                if not region:
                    continue
                x = region.get("x", 0)
                y = region.get("y", 0)
                w = region.get("w", 0)
                h = region.get("h", 0)

                gender_data = face.get("gender", {})
                gender = max(gender_data, key=gender_data.get) if isinstance(gender_data, dict) else str(gender_data)
                age = str(face.get("age", "?"))
                emotion = str(face.get("dominant_emotion", "?"))
                label = f"Gender: {gender} | Age: {age} | Emotion: {emotion}"

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (10, y_label), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                y_label += 30

            analyzed_frame = frame.copy()
        except Exception as e:
            print(" Lỗi phân tích:", e)

        if is_gui_available():
            cv2.imshow("DeepFace webcam", frame)

        key = cv2.waitKey(1)
        if key == 27 or key == ord('e'):
            print("Đã thoát webcam.")
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()

    if analyzed_frame is not None:
        filename = f"deepface_webcam_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, analyzed_frame)
        print(f" Đã lưu ảnh cuối cùng từ webcam: {output_path}")
    else:
        print(" Không có ảnh nào được lưu.")

def main():
    while True:
        print("\n=== DeepFace Face Detection ===")
        print("1. Analyze from image")
        print("2. Analyze from video")
        print("3. Analyze from webcam")
        print("4. Exit")
        choice = input(" Nhập lựa chọn (1/2/3/4): ")

        if choice == '1':
            path = input(" Nhập đường dẫn ảnh: ")
            analyze_from_image(path)
        elif choice == '2':
            path = input("🎞️ Nhập đường dẫn video: ")
            analyze_from_video(path)
        elif choice == '3':
            analyze_from_webcam()
        elif choice == '4':
            print(" Kết thúc chương trình.")
            break
        else:
            print(" Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()




=== DeepFace Face Detection ===
1. Analyze from image
2. Analyze from video
3. Analyze from webcam
4. Exit
👋 Kết thúc chương trình.
